In [1]:
import sys
sys.path.append("./legal-radar/")
sys.path = list(set(sys.path))
import os
os.getcwd()
os.chdir('./legal-radar/')

from legal_radar.services.semantic_search_evaluation import SemanticSearchEvaluation
from legal_radar import config
from legal_radar.services.model_registry import embedding_registry

2021-11-26 11:35:32.061551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-26 11:35:32.061575: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
FAISS_BUCKET_NAME = 'faiss-index'
FAISS_INDEX_FINREG_NAME = 'faiss_index_finreg'
FIN_REG_SPLITTED_ES_INDEX = 'ds_finreg_splitted'
SAMPLE_QUESTIONS_CSV_PATH = 'tests/test_data/sample_questions_v4.csv'
SAMPLE_QUESTIONS_CSV_SEP = ';'

In [5]:
EXPERIMENT_CONFIGS = [
    (10, 5),
    (20, 10),
    (50, 25),
    (100, 50)
]
results = []
for split_window_size, split_window_step in EXPERIMENT_CONFIGS:
    fin_reg_es_index_name = '_'.join(map(str, (FIN_REG_SPLITTED_ES_INDEX, split_window_size, split_window_step)))
    faiss_index_finreg_name = '_'.join(map(str, (FAISS_INDEX_FINREG_NAME, split_window_size, split_window_step, '.pkl')))
    semantic_search_evaluation = SemanticSearchEvaluation(documents_es_index_name = config.EU_FINREG_CELLAR_ELASTIC_SEARCH_INDEX_NAME,
                 splitted_documents_es_index_name = fin_reg_es_index_name,
                 faiss_bucket_name = FAISS_BUCKET_NAME,
                 faiss_index_path = faiss_index_finreg_name,
                 embedding_model = embedding_registry.sent2vec_universal_sent_encoding(),
                 sample_questions_csv_path = SAMPLE_QUESTIONS_CSV_PATH,
                 sample_questions_csv_sep = SAMPLE_QUESTIONS_CSV_SEP)
    results.append(semantic_search_evaluation.evaluate())
    del semantic_search_evaluation

INFO:absl:Using /tmp/tfhub_modules to cache modules.
2021-11-26 11:38:46.171326: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-26 11:38:46.171349: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-26 11:38:46.171363: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (293e399508a7): /proc/driver/nvidia/version does not exist
2021-11-26 11:38:46.171495: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100% (5791 of 5791) |####################| Elapsed Time: 

AttributeError: module 'pandas.io.pickle' has no attribute 'loads'

In [19]:
FAISS_BUCKET_NAME = 'faiss-index'
#FAISS_INDEX_FINREG_NAME = 'faiss_index_finreg.pkl'
FAISS_INDEX_FINREG_NAME = 'faiss_index_finreg'
FIN_REG_SPLITTED_ES_INDEX = 'ds_finreg_splitted'
DATES_DOCUMENT = 'dates_document'
HTML_LINKS = 'htmls_to_download'
DEFAULT_SEARCH = """The Semantic Interoperability Community develops solutions to help European public administrations perform seamless and meaningful cross-border and cross-domain data exchanges."""
TEXT_PIECE = 'text_piece'

In [9]:
def load_documents():
    """Read the data from ES."""
    es_store = store_registry.es_index_store()
    df = es_store.get_dataframe(index_name=config.EU_FINREG_CELLAR_ELASTIC_SEARCH_INDEX_NAME)
    df[DATES_DOCUMENT] = pd.to_datetime(df[DATES_DOCUMENT]).dt.date
    return df


def load_splitted_documents():
    """Read the data from ES."""
    es_store = store_registry.es_index_store()
    return es_store.get_dataframe(index_name=FIN_REG_SPLITTED_ES_INDEX)


def load_emb_model():
    return embedding_registry.sent2vec_universal_sent_encoding()


def load_faiss_index():
    """Load and deserialize the Faiss index."""
    minio_store = store_registry.minio_object_store(minio_bucket=FAISS_BUCKET_NAME)
    data = pickle.loads(minio_store.get_object(object_name=FAISS_INDEX_FINREG_NAME))
    return faiss.deserialize_index(data)


documents = load_documents()
model = load_emb_model()
splitted_documents = load_splitted_documents()
faiss_index = load_faiss_index()


100% (5791 of 5791) |####################| Elapsed Time: 0:00:04 Time:  0:00:04
100% (190656 of 190656) |################| Elapsed Time: 0:01:21 Time:  0:01:21


In [10]:

def semantic_search(user_input: str):
    num_results = 100
    embeddings = model.encode(sentences=[user_input])
    D, I = faiss_index.search(np.array(embeddings).astype("float32"), k=num_results)
    document_parts = pd.DataFrame(splitted_documents.iloc[I.flatten().tolist()])
    document_parts['similarity'] = pd.Series(D.flatten().tolist()).values
    document_parts = document_parts.drop_duplicates(DOCUMENT_ID_SOURCE).reset_index(drop=True)
    documents_id = document_parts[DOCUMENT_ID_SOURCE].values
    result_documents = pd.DataFrame(documents.loc[documents_id]).reset_index(drop=True)
    result_documents['similarity'] = document_parts['similarity']
    result_documents['text_piece'] = document_parts['text_piece']

    return result_documents


In [13]:
# documents, celex number and url reference from sample question file
sample_questions = pd.read_csv('tests/test_data/sample_questions_v4.csv', sep=';')
sample_questions = sample_questions[sample_questions['Questions/Text Extracts'].notnull()]

In [14]:

def find_part_in_search_result(result_set: pd.DataFrame, reference_dataset_celex_number: str,
                               result_set_celex_number: str = 'celex_numbers') -> tuple:
    """Finds the position and the similarity of the documents parts from the result set

    Args:
        result_set (pd.DataFrame): the result dataset from semantic search execution
        reference_dataset_celex_number (str): celex numbers from test bed dataset
        result_set_celex_number (str, optional): [description]. Defaults to 'celex_numbers'.

    Returns:
        tuple: the position and the similarity from document part result set
    """
    reduced_array_dataset = reduce_array_column(result_set, result_set_celex_number).reset_index(drop=True)
    index = reduced_array_dataset[
        reduced_array_dataset[result_set_celex_number].isin([reference_dataset_celex_number])].index.to_list()
    position = reduced_array_dataset['text_piece'].loc[index].index.to_list()
    similarity = reduced_array_dataset['similarity'].apply(lambda x: 1 / (1 + x)).loc[index].to_list()

    return position, similarity


def find_document_in_search_result(result_set: pd.DataFrame, reference_dataset_celex_number: str,
                                   result_set_celex_number: str = 'celex_numbers') -> list:
    """Finds the position and the similarity of the documents from the result set

    Args:
        result_set (pd.DataFrame): the result dataset from semantic search execution
        reference_dataset_celex_number (str): celex numbers from test bed dataset
        result_set_celex_number (str, optional): [description]. Defaults to 'celex_numbers'.

    Returns:
        list: the position of the document from result set
    """
    reduced_array_dataset = reduce_array_column(result_set, result_set_celex_number).reset_index(drop=True)
    index = reduced_array_dataset[
        reduced_array_dataset[result_set_celex_number].isin([reference_dataset_celex_number])].index.to_list()
    position = reduced_array_dataset['title'].loc[index].index.to_list()

    return position


def evaluate_parts(test_bed: pd.DataFrame) -> list:
    """Executes each input query from the test bed dataset into semantic search and grabs the position and
        the similarity of the documents and documents' part of the result set.

    Args:
        test_bed (pd.DataFrame): test dataset with the input queries and comparable celex number

    Returns:
        list: the position and the similarity of the documents and documents' part
    """
    result = []
    for index, row in test_bed.iterrows():
        result_set = semantic_search(row['Questions/Text Extracts'])
        position_p, similarity = find_part_in_search_result(result_set, row['Celex No'])
        position_d = find_document_in_search_result(result_set, row['Celex No'])
        result.append({
            'position_part': position_p,
            'position_document': position_d,
            'similarity': similarity
        })

    return result


def merge_test_bed_with_result_set(test_bed: pd.DataFrame, result_set: list) -> pd.DataFrame:
    """Merge the test bed dataframe and the result set list into a single dataframe

    Args:
        test_bed (pd.DataFrame): test dataset with the input queries and comparable celex number 
        result_set (list): the result from evaluation part

    Returns:
        pd.DataFrame: merged dataframe from test bed and evaluation part
    """
    result = pd.DataFrame(result_set)
    result = result.assign(in_top_5_slices=result['position_part'].apply(lambda x: any(np.array(x) <= 5)),
                           in_top_10_slices=result['position_part'].apply(lambda x: any(np.array(x) <= 10)),
                           in_top_5_documents=result['position_document'].apply(
                               lambda x: any(np.array(x) <= 5)),
                           in_top_10_documents=result['position_document'].apply(
                               lambda x: any(np.array(x) <= 10)),
                           in_q3=result['similarity'].apply(lambda x: any(np.array(x) >= 0.75)))

    return pd.merge(test_bed, result, on=test_bed.index, how="inner")


In [15]:
evaluation = evaluate_parts(sample_questions)
result = merge_test_bed_with_result_set(sample_questions, evaluation)

In [98]:
type(result)

pandas.core.frame.DataFrame

In [50]:
EXPERIMENT_CONFIGS = [
    # (1, 1),
    # (2, 1),
    # (5, 2),
    (10, 5),
    (20, 10),
    (50, 25),
    (100, 50)
]

In [42]:
def reduce_array_column(data_frame: pd.DataFrame, column: str, new_column: str = None):
    """
        assuming that the column contains array objects,
        reduces these arrays to a string of concatenated values
        :data_frame: the pandas DataFrame
        :column: the column with array values
        :new_column: the new column where the concatenated strings are placed;
                     If the new_column is None then the original column is replaced
    """
    def foo_func(x):
        if x is None: 
            print('None type!')
            x = ['INVALID_CELEX_NUMBER']
        return ', '.join(list(x))
    if new_column:
        data_frame[new_column] = data_frame[column].apply(lambda x: foo_func(x))
    else:
        new_column_values = data_frame[column].apply(lambda x: foo_func(x))
        new_data_frame = data_frame.assign(**{column: new_column_values})

    return new_data_frame

In [51]:
es_store = store_registry.es_index_store()
documents = es_store.get_dataframe(index_name=config.EU_FINREG_CELLAR_ELASTIC_SEARCH_INDEX_NAME)
documents[DATES_DOCUMENT] = pd.to_datetime(documents[DATES_DOCUMENT]).dt.date

100% (5791 of 5791) |####################| Elapsed Time: 0:00:04 Time:  0:00:04


In [52]:
EXPERIMENT_CONFIGS = [
    # (1, 1),
    # (2, 1),
    # (5, 2),
    (10, 5),
    (20, 10),
    (50, 25),
    (100, 50)
]
results = []
for split_window_size, split_window_step in EXPERIMENT_CONFIGS:
    fin_reg_es_index_name = '_'.join(map(str, (FIN_REG_SPLITTED_ES_INDEX, split_window_size, split_window_step)))
    faiss_index_finreg_name = '_'.join(
        map(str, (FAISS_INDEX_FINREG_NAME, split_window_size, split_window_step, '.pkl')))
    print(fin_reg_es_index_name, faiss_index_finreg_name)
    splitted_documents = es_store.get_dataframe(index_name=fin_reg_es_index_name)
    minio_store = store_registry.minio_object_store(minio_bucket=FAISS_BUCKET_NAME)
    data = pickle.loads(minio_store.get_object(object_name=faiss_index_finreg_name))
    faiss_index = faiss.deserialize_index(data)
    evaluation = evaluate_parts(sample_questions)
    results.append(merge_test_bed_with_result_set(sample_questions, evaluation))

ds_finreg_splitted_10_5 faiss_index_finreg_10_5_.pkl


100% (190656 of 190656) |################| Elapsed Time: 0:01:21 Time:  0:01:21


ds_finreg_splitted_20_10 faiss_index_finreg_20_10_.pkl


100% (94720 of 94720) |##################| Elapsed Time: 0:00:41 Time:  0:00:41


None type!
None type!
ds_finreg_splitted_50_25 faiss_index_finreg_50_25_.pkl


100% (37910 of 37910) |##################| Elapsed Time: 0:00:17 Time:  0:00:17


None type!
None type!
None type!
None type!
ds_finreg_splitted_100_50 faiss_index_finreg_100_50_.pkl


100% (19625 of 19625) |##################| Elapsed Time: 0:00:11 Time:  0:00:11


None type!
None type!
None type!
None type!
None type!
None type!


In [48]:
RESULT_COLUMNS = [
    'in_top_5_slices',	'in_top_10_slices',
    'in_top_5_documents',	'in_top_10_documents',	'in_q3'
]

In [62]:
results[0]['in_top_5_slices'].value_counts(normalize=True)[1]
{
    "split_type": 0,
    "result_column": 'in_top_5_slices',
    "True": results[0]['in_top_5_slices'].value_counts(normalize=True)[1],
    "False": results[0]['in_top_5_slices'].value_counts(normalize=True)[0]
}

{'split_type': 0,
 'result_column': 'in_top_5_slices',
 'True': 0.8194842406876791,
 'False': 0.18051575931232092}

In [74]:
results[2][result_columns].value_counts(normalize=True).index

Index([False], dtype='object')

In [86]:
result_df = []

for index in range(0,len(EXPERIMENT_CONFIGS)):
    for result_columns in RESULT_COLUMNS:
        tmp = results[index][result_columns].value_counts(normalize=True)
        tmp_dict2 = { str(index):tmp[index] for index in tmp.index}
        tmp_dict1 = {
        "split_type": str(EXPERIMENT_CONFIGS[index]),
        "result_column": result_columns,
        }
        tmp_dict1.update(tmp_dict2)
        result_df.append(tmp_dict1)
result_df = pd.DataFrame(result_df).fillna(0)

In [84]:
import plotly.express as px

In [97]:
fig = px.bar(result_df, x="split_type", y="True", color="result_column",barmode='group', title="Search results based on window size")
fig.update_yaxes(range=[0, 1])
fig.show()
fig = px.bar(result_df, x="split_type", y="False", color="result_column",barmode='group', title="Search results based on window size")
fig.show()